Front Matter

In [1]:
import os
import pandas as pd
from tqdm import tqdm
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
import joblib


Sector Definition

In [ ]:
# Define the sectors and their top 10 performing stocks
sectors = {
    "technology": ["AAPL", "MSFT", "GOOGL", "NVDA", "META", "ORCL", "ADBE", "CRM", "INTC", "AMD", "IBM"],
    "medicine": ["PFE", "JNJ", "MRK", "ABBV", "BMY", "LLY", "AMGN", "GILD", "REGN", "BIIB"],
    "industrial": ["GE", "CAT", "HON", "BA", "MMM", "DE", "LMT", "RTX", "NOC", "EMR"],
    "finance": ["JPM", "BAC", "WFC", "C", "GS", "MS", "AXP", "BK", "USB", "PNC"],
    "consumer_discretionary": ["TSLA", "AMZN", "HD", "LOW", "NKE", "SBUX", "MCD", "TGT", "DIS", "ROST"]
}


Data Retrival and CSV Creation

In [ ]:
# Alpha Vantage API key
key = open('AV_API_Key.txt').read().strip()

# Function to request stock price data and save to CSV
def request_and_save_stock_data(symbol, sector, token):
    q_string = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={}&outputsize=full&apikey={}'
    
    # Request data from Alpha Vantage
    print(f"Retrieving data for {symbol}...")
    r = requests.get(q_string.format(symbol, token))
    if "Time Series (Daily)" not in r.json():
        print(f"Error retrieving data for {symbol}")
        return None

    # Parse data into a DataFrame
    date = []
    df = pd.DataFrame()
    for i in r.json()['Time Series (Daily)'].keys():
        date.append(i)
        row = pd.DataFrame.from_dict(r.json()['Time Series (Daily)'][i], orient='index').reset_index().T[1:]
        df = pd.concat([df, row], ignore_index=True)
    df.columns = ["open", "high", "low", "close", "volume"]
    df['date'] = date

    # Save to CSV
    file_name = f"{sector}_{symbol}.csv"
    df.to_csv(file_name, index=False)
    print(f"Data saved to {file_name}")
    return file_name

# Iterate over sectors and stocks to save data
stock_csv_mapping = {}
for sector, stocks in tqdm(sectors.items()):
    stock_csv_mapping[sector] = {}
    for stock in stocks:
        file_name = request_and_save_stock_data(stock, sector, key)
        if file_name:
            stock_csv_mapping[sector][stock] = file_name

# Output the mapping of stocks to their CSV files
stock_csv_mapping

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving data for AAPL...
Data saved to technology_AAPL.csv
Retrieving data for MSFT...
Data saved to technology_MSFT.csv
Retrieving data for GOOGL...
Data saved to technology_GOOGL.csv
Retrieving data for NVDA...
Data saved to technology_NVDA.csv
Retrieving data for META...
Data saved to technology_META.csv
Retrieving data for ORCL...
Data saved to technology_ORCL.csv
Retrieving data for ADBE...
Data saved to technology_ADBE.csv
Retrieving data for CRM...
Data saved to technology_CRM.csv
Retrieving data for INTC...
Data saved to technology_INTC.csv
Retrieving data for AMD...


 20%|██        | 1/5 [09:43<38:55, 583.95s/it]

Data saved to technology_AMD.csv
Retrieving data for PFE...
Data saved to medicine_PFE.csv
Retrieving data for JNJ...
Data saved to medicine_JNJ.csv
Retrieving data for MRK...
Data saved to medicine_MRK.csv
Retrieving data for ABBV...
Data saved to medicine_ABBV.csv
Retrieving data for BMY...
Data saved to medicine_BMY.csv
Retrieving data for LLY...


Data Preprocessing

In [ ]:
# Perform basic data preprocessing on all CSV files
preprocessed_data = {}

for sector, stocks in stock_csv_mapping.items():
    preprocessed_data[sector] = {}
    for stock, file_name in stocks.items():
        # Load the CSV file
        df = pd.read_csv(file_name)
        
        # Convert date column to datetime
        df['date'] = pd.to_datetime(df['date'])
        
        # Convert columns to numeric
        for col in ['open', 'high', 'low', 'close', 'volume']:
            df[col] = pd.to_numeric(df[col])
        
        # Replace NaN values with column mean
        df.fillna(df.select_dtypes(include=['number']).mean(), inplace=True)
        
        # Calculate technical indicators
        # Moving Averages
        ma_day = [10, 50, 100]
        for ma in ma_day:
            column_name = f"MA for {ma} days"
            df[column_name] = df['close'].rolling(window=ma).mean()
        
        # Daily Return
        df['Daily Return'] = df['close'].pct_change()
        
        # EMAs for various periods
        df['ema7'] = df['close'].ewm(span=7, adjust=False).mean()
        df['ema14'] = df['close'].ewm(span=14, adjust=False).mean()
        df['ema30'] = df['close'].ewm(span=30, adjust=False).mean()
        df['ema60'] = df['close'].ewm(span=60, adjust=False).mean()

        # Sort by date and reset index
        df = df.sort_values('date')
        df.reset_index(drop=True, inplace=True)
        
        # Fill any remaining NaN values
        df.fillna(method='bfill', inplace=True)
        
        # Save the preprocessed DataFrame back to the same CSV file
        df.to_csv(file_name, index=False)
        
        # Store the preprocessed DataFrame in the dictionary
        preprocessed_data[sector][stock] = df

# Output the first few rows of a sample preprocessed stock
sample_sector = list(preprocessed_data.keys())[0]
sample_stock = list(preprocessed_data[sample_sector].keys())[0]
preprocessed_data[sample_sector][sample_stock].head()

Sector Wise Visualizations

In [ ]:
# Visualize sector-wise stock data
for sector, stocks in preprocessed_data.items():
    print(f"\nVisualizing data for sector: {sector}\n")
    
    # Combine all stocks in the sector into one DataFrame for visualization
    combined_df = pd.DataFrame()
    for stock, df in stocks.items():
        df_temp = df.copy()
        df_temp['stock'] = stock  # Add a column to identify the stock
        combined_df = pd.concat([combined_df, df_temp], ignore_index=True)

    # Plot closing prices for all stocks in the sector
    plt.figure(figsize=(14, 7))
    sns.lineplot(data=combined_df, x='date', y='close', hue='stock')
    plt.title(f"Closing Prices for {sector.capitalize()} Sector")
    plt.xlabel("Date")
    plt.ylabel("Closing Price")
    plt.legend(title="Stock")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Plot daily returns distribution
    plt.figure(figsize=(14, 7))
    for stock, df in stocks.items():
        sns.kdeplot(df['Daily Return'].dropna(), label=stock)
    plt.title(f"Daily Returns Distribution for {sector.capitalize()} Sector")
    plt.xlabel("Daily Return")
    plt.ylabel("Density")
    plt.legend(title="Stock")
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    
    # Plot EMA comparison for a representative stock
    sample_stock = list(stocks.keys())[0]
    plt.figure(figsize=(14, 7))
    plt.plot(stocks[sample_stock]['date'], stocks[sample_stock]['close'], label='Close Price')
    plt.plot(stocks[sample_stock]['date'], stocks[sample_stock]['ema7'], label='EMA 7 days')
    plt.plot(stocks[sample_stock]['date'], stocks[sample_stock]['ema30'], label='EMA 30 days')
    plt.plot(stocks[sample_stock]['date'], stocks[sample_stock]['ema60'], label='EMA 60 days')
    plt.title(f"EMA Comparison for {sample_stock} in {sector.capitalize()} Sector")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

BiLSTM Model Implementation

In [ ]:
# Split data into training and testing sets
def split_data(df, test_size=0.2):
    train_size = int(len(df) * (1 - test_size))
    train_data = df[:train_size]
    test_data = df[train_size:]
    return train_data, test_data

# Prepare data for LSTM model
def prepare_data_for_lstm(df, target_col='close', lookback=60):
    """Prepare data for LSTM model with a lookback period"""
    # Scale the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df[target_col].values.reshape(-1, 1))
    
    X, y = [], []
    for i in range(lookback, len(scaled_data)):
        X.append(scaled_data[i-lookback:i, 0])
        y.append(scaled_data[i, 0])
    
    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    
    return X, y, scaler

# Create Bidirectional LSTM model for improved performance
def create_bilstm_model(input_shape):
    """Create Bidirectional LSTM model for stock price prediction"""
    model = Sequential()
    model.add(Bidirectional(LSTM(units=50, return_sequences=True, input_shape=input_shape)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(units=50, return_sequences=False)))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Train and evaluate the model
def train_lstm_model(df, lookback=60, epochs=12, batch_size=32, test_size=0.2):
    """Train and evaluate LSTM model"""
    # Split data
    train_data, test_data = split_data(df, test_size)
    
    # Prepare data for LSTM
    X_train, y_train, scaler = prepare_data_for_lstm(train_data, lookback=lookback)
    X_test, y_test, _ = prepare_data_for_lstm(pd.concat([train_data.iloc[-lookback:], test_data]), lookback=lookback)
    
    # Create and train model
    model = create_bilstm_model((X_train.shape[1], 1))
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, 
                        validation_data=(X_test, y_test), verbose=1)
    
    # Predictions
    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)
    
    # Inverse scaling
    train_predictions = scaler.inverse_transform(train_predictions)
    y_train_actual = scaler.inverse_transform(y_train.reshape(-1, 1))
    test_predictions = scaler.inverse_transform(test_predictions)
    y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))
    
    return model, scaler, test_predictions, y_test_actual, test_data['date'].reset_index(drop=True), history

# EMA constraint function
def apply_ema_constraint(prediction, df, window=60, max_deviation=0.1):
    """Apply EMA constraint to ensure prediction is within reasonable range"""
    # Calculate EMA of actual prices
    ema = df['close'].ewm(span=window, adjust=False).mean().iloc[-1]
    
    # Set upper and lower bounds based on EMA
    upper_bound = ema * (1 + max_deviation)
    lower_bound = ema * (1 - max_deviation)
    
    # Apply constraints
    if prediction > upper_bound:
        constrained_prediction = upper_bound
    elif prediction < lower_bound:
        constrained_prediction = lower_bound
    else:
        constrained_prediction = prediction
    
    return constrained_prediction, ema, lower_bound, upper_bound

Training and Saving Models for each Stock

In [ ]:
    # Train and save LSTM models for each stock
trained_models = {}

for sector, stocks in preprocessed_data.items():
    trained_models[sector] = {}
    for stock, df in stocks.items():
        print(f"\nTraining LSTM model for {stock} in {sector} sector...")
        
        # Prepare data
        lookback = 60
        if len(df) <= lookback:
            print(f"Not enough data for {stock}. Skipping...")
            continue
        
        # Train model
        model, scaler, test_predictions, y_test_actual, test_dates, history = train_lstm_model(
            df, lookback=lookback, epochs=12, batch_size=32
        )
        
        # Plot actual vs predicted prices for test set
        plt.figure(figsize=(12, 5))
        plt.plot(test_dates, y_test_actual, label='Actual Prices')
        plt.plot(test_dates, test_predictions, label='Predicted Prices')
        plt.title(f'Stock Price Prediction for {stock}')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend()
        plt.grid(True)
        plt.show()
        
        # Save model and scaler
        model_name = f"{sector}_{stock}_lstm_model.keras"
        scaler_name = f"{sector}_{stock}_scaler.joblib"
        model.save(model_name)
        joblib.dump(scaler, scaler_name)
        
        trained_models[sector][stock] = {
            "model_file": model_name,
            "scaler_file": scaler_name
        }
        
        print(f"Model and scaler saved for {stock}.")

# Output the trained models dictionary
trained_models


Next Day Price Prediction for All Stocks

In [ ]:
# Function to load a saved model and scaler
def load_model_and_scaler(model_file, scaler_file):
    model = load_model(model_file)
    scaler = joblib.load(scaler_file)
    return model, scaler

# Function to predict next day's price
def predict_next_day(model, df, scaler, lookback=60, ema_window=60, max_deviation=0.1):
    """Predict next day's stock price and apply EMA constraint"""
    # Get the last 'lookback' days of data
    last_sequence = df['close'].values[-lookback:]
    
    # Scale the data
    last_sequence = scaler.transform(last_sequence.reshape(-1, 1))
    
    # Reshape for LSTM input
    last_sequence = np.reshape(last_sequence, (1, lookback, 1))
    
    # Predict
    next_day_scaled = model.predict(last_sequence)
    
    # Inverse transform
    next_day_price = scaler.inverse_transform(next_day_scaled)[0, 0]
    
    # Apply EMA constraint
    constrained_price, ema, lower_bound, upper_bound = apply_ema_constraint(
        next_day_price, df, window=ema_window, max_deviation=max_deviation
    )
    
    return constrained_price, next_day_price, ema, lower_bound, upper_bound

# Predict next day prices for all stocks
predictions = {}

for sector, stocks in trained_models.items():
    predictions[sector] = {}
    print(f"\n=== Predictions for {sector.upper()} sector ===\n")
    
    for stock, model_info in stocks.items():
        # Load model and scaler
        model, scaler = load_model_and_scaler(model_info["model_file"], model_info["scaler_file"])
        
        # Get the stock data
        df = preprocessed_data[sector][stock]
        
        # Predict next day price
        constrained_price, raw_prediction, ema, lower_bound, upper_bound = predict_next_day(
            model, df, scaler, lookback=60, ema_window=60, max_deviation=0.1
        )
        
        # Store prediction
        predictions[sector][stock] = {
            "current_price": df['close'].iloc[-1],
            "raw_prediction": raw_prediction,
            "constrained_prediction": constrained_price,
            "ema_60": ema,
            "lower_bound": lower_bound,
            "upper_bound": upper_bound
        }
        
        # Print prediction
        print(f"{stock}:")
        print(f"  Current price: ${df['close'].iloc[-1]:.2f}")
        print(f"  Raw prediction: ${raw_prediction:.2f}")
        print(f"  EMA (60 days): ${ema:.2f}")
        print(f"  Allowed range: ${lower_bound:.2f} to ${upper_bound:.2f}")
        print(f"  Final prediction: ${constrained_price:.2f}")
        print(f"  Expected change: ${constrained_price - df['close'].iloc[-1]:.2f} ({(constrained_price - df['close'].iloc[-1]) / df['close'].iloc[-1] * 100:.2f}%)")
        print()

# Create a summary DataFrame of all predictions
prediction_summary = pd.DataFrame(columns=[
    'Sector', 'Stock', 'Current Price', 'Predicted Price', 'Expected Change', 'Expected Change (%)'
])

row = 0
for sector, stocks in predictions.items():
    for stock, pred in stocks.items():
        prediction_summary.loc[row] = [
            sector,
            stock,
            pred['current_price'],
            pred['constrained_prediction'],
            pred['constrained_prediction'] - pred['current_price'],
            (pred['constrained_prediction'] - pred['current_price']) / pred['current_price'] * 100
        ]
        row += 1

# Sort by expected change percentage
prediction_summary = prediction_summary.sort_values('Expected Change (%)', ascending=False)
prediction_summary


Visualizing Prediction Results

In [ ]:
# Create visualization of predicted changes across all stocks
plt.figure(figsize=(15, 8))
sns.barplot(data=prediction_summary, x='Stock', y='Expected Change (%)', hue='Sector')
plt.title('Predicted Price Changes for Next Trading Day')
plt.xlabel('Stock')
plt.ylabel('Expected Change (%)')
plt.xticks(rotation=90)
plt.axhline(y=0, color='r', linestyle='-')
plt.tight_layout()
plt.show()

# Create visualization of sector-wise average predictions
sector_avg = prediction_summary.groupby('Sector')['Expected Change (%)'].mean().reset_index()
plt.figure(figsize=(12, 6))
sns.barplot(data=sector_avg, x='Sector', y='Expected Change (%)')
plt.title('Average Predicted Change by Sector')
plt.xlabel('Sector')
plt.ylabel('Average Expected Change (%)')
plt.axhline(y=0, color='r', linestyle='-')
plt.tight_layout()
plt.show()

# Create a price range visualization for top 10 stocks
top_10 = prediction_summary.head(10)
plt.figure(figsize=(14, 7))
for i, row in top_10.iterrows():
    sector = row['Sector']
    stock = row['Stock']
    current = row['Current Price']
    predicted = row['Predicted Price']
    pred_info = predictions[sector][stock]
    
    plt.scatter(stock, current, color='blue', s=100, label='Current Price' if i==0 else "")
    plt.scatter(stock, predicted, color='green', s=100, label='Predicted Price' if i==0 else "")
    plt.plot([stock, stock], [pred_info['lower_bound'], pred_info['upper_bound']], 
             color='red', linestyle='-', label='Allowed Range' if i==0 else "")
    
plt.title('Price Prediction with EMA Constraints for Top 10 Stocks')
plt.xlabel('Stock')
plt.ylabel('Price ($)')
plt.legend()
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

NOVELTY: Bayesian Networks to evaluate External Market Factors

Front Matter 2

In [ ]:
import yfinance as yf
import pandas as pd
from fredapi import Fred
import datetime
import seaborn as sns
import matplotlib.pyplot as plt

Data Collection and Pre-Processing

In [ ]:
# Initialize FRED API (use your API key)
fred = Fred(api_key='YOUR_FRED_API_KEY')

# Define date range
start_date = '2010-01-01'
end_date = '2023-12-31'

# IBM stock data
ibm = yf.download('IBM', start=start_date, end=end_date)  # IBM stock price & volume
print("IBM Columns:", ibm.columns)

# IBM Option Chain for Implied Volatility
ticker = yf.Ticker('IBM')
options_dates = ticker.options
option_chain = ticker.option_chain(options_dates[0])  # Take nearest expiry
ibm_iv = option_chain.calls['impliedVolatility'].mean()  # Average IV from calls

# Market index (S&P 500)
sp500 = yf.download('^GSPC', start=start_date, end=end_date)  # Market Index Return

# Sector ETF (Technology ETF XLK)
xlk = yf.download('XLK', start=start_date, end=end_date)  # Sector Performance Index

# VIX index (daily volatility measure)
vix = yf.download('^VIX', start=start_date, end=end_date)  # VIX (CBOE Volatility Index)

# Credit Spread proxy using BAA - AAA spread from FRED
baa_yield = fred.get_series('BAA10Y', observation_start=start_date, observation_end=end_date)  # BAA corporate bonds
aaa_yield = fred.get_series('AAA10Y', observation_start=start_date, observation_end=end_date)  # AAA corporate bonds
credit_spread = (baa_yield - aaa_yield).resample('D').ffill()  # Credit Spreads

# 10-Year Treasury Yield from FRED
treasury_yield = fred.get_series('GS10', observation_start=start_date, observation_end=end_date)  # Risk-free rate
treasury_yield = treasury_yield.resample('D').ffill()

# CPI data from FRED
cpi = fred.get_series('CPIAUCSL', observation_start=start_date, observation_end=end_date)  # Inflation proxy
cpi = cpi.resample('D').ffill()

# Align all data to the same date index
combined_data = pd.DataFrame(index=ibm.index)
combined_data['IBM_Close'] = ibm['Close']  # IBM Close
combined_data['IBM_Volume'] = ibm['Volume']  # Trading Volume
combined_data['SP500_Close'] = sp500['Close']  # Market Index Return
combined_data['XLK_Close'] = xlk['Close']  # Sector Performance Index
combined_data['VIX'] = vix['Close']  # VIX
combined_data['Treasury_Yield'] = treasury_yield.reindex(ibm.index)  # Treasury Yield
combined_data['CPI'] = cpi.reindex(ibm.index)  # CPI
combined_data['Credit_Spread'] = credit_spread.reindex(ibm.index)  # Credit Spread
combined_data['Implied_Volatility'] = ibm_iv  # Static IV for now

# Drop rows with missing values
combined_data.dropna(inplace=True)

Correlation Score Calculation

In [ ]:
# Calculate IBM daily returns
combined_data['IBM_Returns'] = combined_data['IBM_Close'].pct_change()

# Calculate company-specific features directly from combined_data
combined_data['IBM_Volatility'] = combined_data['IBM_Close'].pct_change().rolling(window=30).std()  # Historical Volatility
combined_data['IBM_Momentum'] = combined_data['IBM_Close'] / combined_data['IBM_Close'].shift(20) - 1  # Price Momentum (20-day)

# Calculate daily returns for other relevant factors
combined_data['SP500_Returns'] = combined_data['SP500_Close'].pct_change()
combined_data['XLK_Returns'] = combined_data['XLK_Close'].pct_change()
combined_data['VIX_Change'] = combined_data['VIX'].pct_change()
combined_data['Treasury_Yield_Change'] = combined_data['Treasury_Yield'].pct_change()
combined_data['CPI_Change'] = combined_data['CPI'].pct_change()
combined_data['Credit_Spread_Change'] = combined_data['Credit_Spread'].pct_change()
combined_data['Volume_Change'] = combined_data['IBM_Volume'].pct_change()

# Drop NA after pct_change
combined_data.dropna(inplace=True)

# Calculate threshold values for each factor based on percentiles
thresholds = {}
for factor in [
    'SP500_Returns', 'XLK_Returns', 'VIX_Change', 'Treasury_Yield_Change',
    'CPI_Change', 'Credit_Spread_Change', 'IBM_Volatility', 'IBM_Momentum',
    'Volume_Change', 'Implied_Volatility'
]:
    if factor == 'Implied_Volatility':
        thresholds[factor] = [combined_data[factor].mean() * 0.85, combined_data[factor].mean() * 1.15]
    else:
        thresholds[factor] = [
            combined_data[factor].quantile(0.33),
            combined_data[factor].quantile(0.66)
        ]

# Calculate correlation matrix
correlation_matrix = combined_data[[
    'IBM_Returns', 'SP500_Returns', 'XLK_Returns', 'VIX_Change', 
    'Treasury_Yield_Change', 'CPI_Change', 'Credit_Spread_Change',
    'IBM_Volatility', 'IBM_Momentum', 'Volume_Change', 'Implied_Volatility'
]].corr()

# Show correlations specifically to IBM Returns
ibm_corr = correlation_matrix['IBM_Returns'].drop('IBM_Returns')

print("Correlation of external factors with IBM Returns:")
print(ibm_corr)

# Optional: Plot heatmap
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()


Construct Conditional Probability Table for External Factors

In [ ]:
def get_cpt_value(value, thresholds, correlation):
    shift = abs(correlation) * 0.25
    base = 1 / 3
    if value < thresholds[0]:
        return base - shift if correlation > 0 else base + shift
    elif value > thresholds[1]:
        return base + shift if correlation > 0 else base - shift
    else:
        return base

# Use most recent values for CPT evaluation
latest = combined_data.iloc[-1]

# Compute conditional probabilities
cp_values = {}
for factor, corr in ibm_corr.items():
    val = latest[factor]
    t = thresholds[factor]
    cp = get_cpt_value(val, t, corr)
    cp_values[factor] = cp
    print(f"{factor}: value={val:.4f}, correlation={corr:.3f}, CP={cp:.3f}")

# Compute overall conditional probability (product of independent CPs)
risk_factor = 1
for cp in cp_values.values():
    risk_factor *= cp

print("\nFinal Conditional Risk Factor for IBM:", risk_factor)

# Example
predicted_price = predictions['technology']['IBM']['constrained_prediction']
adjusted_price = predicted_price * risk_factor
print("Predicted Price:", predicted_price)
print("Adjusted Price after Risk Assessment:", adjusted_price)
